<a href="https://colab.research.google.com/github/FabianaAlbuquerque97/Quantization/blob/main/Exemplo_3_Quantiza%C3%A7%C3%A3o_Bin%C3%A1ria_e_Tern%C3%A1ria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Quantização Binária e Ternária**

Fabiana de Albuquerque

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [ ]:
# Definindo um modelo simples de rede neural convolucional (CNN)
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = x.view(-1, 64 * 7 * 7)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
# Função de quantização binária (Usa a função torch.sign() para converter os valores em -1 e 1.)
def binary_quantization(tensor):
    return torch.sign(tensor)

In [ ]:
# Função de quantização ternária (Usa torch.where() para converter os valores em -1, 0 e 1.)
def ternary_quantization(tensor):
    return torch.where(tensor > 0.5, torch.tensor(1.0), torch.where(tensor < -0.5, torch.tensor(-1.0), torch.tensor(0.0)))


In [ ]:
# Preparando os dados MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)


Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 15.9MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 484kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.45MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 5.80MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# Inicializando o modelo e o otimizador
model = SimpleCNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Treinamento do modelo
for epoch in range(5):  # número de épocas
    for inputs, labels in trainloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Época {epoch + 1}, Perda: {loss.item()}')

Época 1, Perda: 0.11838977038860321
Época 2, Perda: 0.025715842843055725
Época 3, Perda: 0.0035569036845117807
Época 4, Perda: 0.0006961012841202319
Época 5, Perda: 0.07580548524856567


In [ ]:
# Aplicando a quantização
model.eval()
with torch.no_grad():
    for inputs, _ in trainloader:
        outputs = model(inputs)
        binary_outputs = binary_quantization(outputs)
        ternary_outputs = ternary_quantization(outputs)

# Exibindo os resultados
print("Saídas binárias:", binary_outputs)
print("Saídas ternárias:", ternary_outputs)

Saídas binárias: tensor([[-1., -1., -1., -1., -1.,  1.,  1., -1.,  1.,  1.],
        [-1., -1., -1.,  1., -1.,  1., -1., -1.,  1., -1.],
        [ 1., -1., -1., -1., -1.,  1.,  1., -1.,  1., -1.],
        [-1., -1., -1., -1.,  1., -1., -1., -1., -1.,  1.],
        [-1.,  1., -1., -1.,  1., -1., -1.,  1., -1., -1.],
        [-1., -1., -1.,  1., -1.,  1., -1., -1., -1.,  1.],
        [-1., -1., -1., -1.,  1., -1., -1.,  1., -1.,  1.],
        [-1.,  1., -1., -1.,  1., -1., -1., -1., -1., -1.],
        [ 1., -1., -1., -1., -1., -1., -1., -1., -1.,  1.],
        [-1., -1., -1., -1.,  1.,  1., -1., -1.,  1.,  1.],
        [-1., -1., -1.,  1., -1.,  1., -1., -1., -1., -1.],
        [-1., -1., -1.,  1., -1., -1., -1.,  1., -1.,  1.],
        [-1., -1., -1., -1.,  1., -1., -1., -1., -1.,  1.],
        [-1., -1., -1., -1.,  1.,  1., -1., -1., -1.,  1.],
        [-1., -1., -1., -1.,  1., -1., -1.,  1., -1., -1.],
        [-1., -1.,  1.,  1., -1.,  1., -1., -1.,  1.,  1.],
        [-1.,  1., -1.,